In [1]:
import sqlite3
import os

os.makedirs("data/db",exist_ok=True)

In [2]:
connection = sqlite3.connect('data/db/company.db')
cursor = connection.cursor()

In [3]:
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
               (id INTEGER PRIMARY KEY,name TEXT,role TEXT,department TEXT,salary REAL)''')

In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
               (id INTEGER PRIMARY KEY,name TEXT,status TEXT,budget REAL,lead_id INTEGER)''')

In [5]:
employees = [
    (1, 'Alice Johnson', 'Software Engineer', 'Engineering', 85000),
    (2, 'Bob Smith', 'Senior Software Engineer', 'Engineering', 105000),
    (3, 'Carol Davis', 'Project Manager', 'Product', 95000),
    (4, 'David Lee', 'Data Analyst', 'Analytics', 78000),
    (5, 'Eva Martinez', 'HR Manager', 'Human Resources', 72000)
]

projects = [
    (1, 'Website Redesign', 'In Progress', 50000, 3),
    (2, 'Mobile App Development', 'Planned', 120000, 2),
    (3, 'Data Pipeline Upgrade', 'Completed', 75000, 4),
    (4, 'Employee Onboarding System', 'In Progress', 30000, 1),
    (5, 'HR Analytics Dashboard', 'Planned', 40000, 5)
]

In [7]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)',employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)',projects)

In [8]:
connection.commit()
connection.close()

In [9]:
from langchain_community.document_loaders import SQLDatabaseLoader
from langchain_community.utilities import SQLDatabase

d:\AI\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
db = SQLDatabase.from_uri("sqlite:///data/db/company.db")

In [16]:
from typing import List
from langchain_core.documents import Document

def sql_to_documents(db_path:str) -> List[Document] :
    """converts sql db to documents with context"""
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    documents = []
    #create document for each table 
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ;") 
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]

        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        col_names = [col[1] for col in columns]


        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()


        table_content = f"Table : {table_name}"
        table_content += f"Columns : {', '.join(col_names)}\n"
        table_content += f"Total records:  {len(rows)}\n\n"

        table_content += "Sample records: \n"
        for row in rows[:5]:
            record = dict(zip(col_names,row))
            table_content += f"{record}\n"

        doc = Document(
            page_content=table_content,
            metadata = {
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type' : 'sql_table'
            }
        )

        documents.append(doc)

    return documents

In [17]:
sql_to_documents("data/db/company.db")

[Document(metadata={'source': 'data/db/company.db', 'table_name': 'employees', 'num_records': 5, 'data_type': 'sql_table'}, page_content="Table : employeesColumns : id, name, role, department, salary\nTotal records:  5\n\nSample records: \n{'id': 1, 'name': 'Alice Johnson', 'role': 'Software Engineer', 'department': 'Engineering', 'salary': 85000.0}\n{'id': 2, 'name': 'Bob Smith', 'role': 'Senior Software Engineer', 'department': 'Engineering', 'salary': 105000.0}\n{'id': 3, 'name': 'Carol Davis', 'role': 'Project Manager', 'department': 'Product', 'salary': 95000.0}\n{'id': 4, 'name': 'David Lee', 'role': 'Data Analyst', 'department': 'Analytics', 'salary': 78000.0}\n{'id': 5, 'name': 'Eva Martinez', 'role': 'HR Manager', 'department': 'Human Resources', 'salary': 72000.0}\n"),
 Document(metadata={'source': 'data/db/company.db', 'table_name': 'projects', 'num_records': 5, 'data_type': 'sql_table'}, page_content="Table : projectsColumns : id, name, status, budget, lead_id\nTotal record